In [19]:
from docx import Document


def extract_text_from_docx(docx_file,start_paragraph, end_paragraph):


    #Load .docx file 
    doc = Document(docx_file) 
    extracted_text = [] 
    
    start_index =  start_paragraph - 1 
    end_index = end_paragraph

    for i,para in enumerate(doc.paragraphs):
        if start_index <= i < end_index:
            extracted_text.append(para.text)


    return '\n'.join(extracted_text)




start_para = 7
end_para = 10
file_path = 'doc_data/data_1.docx' 
text = extract_text_from_docx(file_path,start_para,end_para)

print(text)


๑.	เรื่องเดิม
	ตามบันทึก สำนักงาน กสทช. เขต 23 งานตรวจสอบและกำกับดูแล ที่ สทช 2303.3/19   ลงวันที่ 22 มกราคม 2567 เรื่อง แผนการปฏิบัติงานตรวจสอบและกำกับดูแลประจำปี 2567 ได้รับความเห็นชอบแผนปฏิบัติงานตรวจสอบและกำกับดูแลประจำปี ๒๕๖7 ของสำนักงาน กสทช. เขต ๒๓ โดย ผภภ. 23 รายละเอียดปรากฏตามเอกสารแนบ นั้น 
๒.	เรื่องเพื่อพิจารณา
	2.1	เพื่อให้การปฏิบัติงานตามข้อ ๑. เป็นไปด้วยความเรียบร้อย จึงขออนุมัติให้พนักงาน ภภ. 23 เดินทางไปปฏิบัติหน้าที่ ระหว่างวันที่ 24 – 25 มกราคม 2567  ในเขตพื้นที่จังหวัดชัยภูมิ และพื้นที่ใกล้เคียง โดยมีพนักงานดังนี้
